# 16. Preprocesado: revisión de la clase YSO

In [1]:
# Configuración general para evitar errores de warnings y compatibilidad
import warnings
import os
warnings.filterwarnings("ignore")
os.environ["RICH_NO_RICH"] = "1"
print("Configuración de entorno aplicada.")

Configuración de entorno aplicada.


## NUEVO PREPROCESADO GUARDANDO IDs DE OBJETOS

Luego se pueden usar para filtrar curvas dudosas que no están clasificando bien, y así depurar el entrenamiento.

In [1]:
import warnings
import numpy as np
import sys
from pathlib import Path

# Añadir la raíz del proyecto al path
ROOT_DIR = Path.cwd().parent  # <- sube un nivel para alcanzar la raíz del proyecto
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

# Ignorar solo los RuntimeWarning de numpy (como overflows en reduce)
warnings.filterwarnings("ignore", category=RuntimeWarning, module="numpy")

from src.fase2.script_1_transformer_preprocessing_optimizado_2 import main as preprocessing_optimized_with_features

max_per_class_override={
    "Irregular": 9000,
    "Rotational": 9000,
    "Eclipsing Binary": 9000,
    "Delta Scuti": None,            # 7.550 → TODAS
    "RR Lyrae": 9000,               # 41.208 → TODAS NO
    "Young Stellar Object": None,   # 9.809 → TODAS
    "Cataclysmic": None,            # 2.080 → TODAS
    "White Dwarf": 0,               # 0 → LA ELIMINAMOS
    "Variable": 0                   # 0 → LA ELIMINAMOS
}

preprocessing_optimized_with_features(
    seq_length=25000,
    max_per_class=None, # usamos override completo
    max_per_class_override=max_per_class_override,
    parquet_batch_size=10_000_000,
    dataloader_batch_size=128,
    num_workers=20,
    #errores_csv_path=Path("../outputs/errores_mal_clasificados.csv")
)

📂 Cargando datos en lotes con PyArrow...
💾 [INFO] Cargando agrupación de curvas desde cache: /home/ec2-user/SageMaker/astro_transformer/src/fase2/../../data/train/grouped_data.pkl
✅ [INFO] Agrupación cargada desde cache. Total objetos: 55439
⏳ [INFO] Tiempo en agrupación de datos: 10.9 segundos
🚀 Procesando 55439 curvas en paralelo usando 20 CPUs...
⏳ [INFO] Tiempo en procesamiento paralelo: 82.2 segundos
🔋 [INFO] Curvas válidas tras filtrado: 55342

🔍 Realizando prueba rápida en características auxiliares...
✅ Sample 0 sin problemas: [-0.54765874 -0.46691176 -0.58238172 -0.45465844 -0.48360653 -0.18917597
  0.26550589]
✅ Sample 1 sin problemas: [-0.31123622 -0.23529412 -0.42088094 -0.30324447 -0.22950819  0.35237014
 -0.16461357]
✅ Sample 2 sin problemas: [-0.50290883 -0.43382352 -0.54812398 -0.73793606 -0.4426229  -0.35511186
 -0.06430592]
✅ Sample 3 sin problemas: [ 1.37089379  1.63602933  0.83931489 -1.24292777  1.21311469 -1.06736809
 -0.27657237]
✅ Sample 4 sin problemas: [-0.131

(<torch.utils.data.dataloader.DataLoader at 0x7f1cae76d5d0>,
 <torch.utils.data.dataloader.DataLoader at 0x7f1cae76e3b0>)

**Con el siguiente script intentamos detectar los IDs de las curvas YSO que están confundiendo al modelo**.

En la matriz de confusión vimos muchos casos de clase predicha YSO que en realidad era cualquiera de las otras. 

In [3]:
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.serialization
from torch.utils.data import DataLoader
import sys
import gc
import os
import argparse
import warnings
from pathlib import Path

# Añadir la raíz del proyecto al path
ROOT_DIR = Path.cwd().parent  # <- sube un nivel para alcanzar la raíz del proyecto
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

# Ignorar solo los RuntimeWarning de numpy (como overflows en reduce)
warnings.filterwarnings("ignore", category=RuntimeWarning, module="numpy")

from src.fase2.script_2_transformer_fine_tuning_optimizado import AstroConformerClassifier as AstroConformerClassifier, evaluate

# Detectar dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"

# acciones para resolver los problemas de memoria
# 1. Liberar memoria
gc.collect()
torch.cuda.empty_cache()

# 2. Optimizar fragmentacion
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

torch.backends.cudnn.benchmark = True

def detectar_yso_confundidas(preds, true, ids, label_encoder):
    """
    Retorna un DataFrame con los objetos cuya clase fue predicha como YSO,
    pero su clase real era diferente.
    """
    inv_label_encoder = {v: k for k, v in label_encoder.items()}

    errores = []
    for pred, real, obj_id in zip(preds, true, ids):
        clase_pred = inv_label_encoder[pred]
        clase_real = inv_label_encoder[real]
        if clase_pred == "Young Stellar Object" and clase_real != "Young Stellar Object":
            errores.append({"id_objeto": obj_id, "clase_real": clase_real, "clase_predicha": clase_pred})

    return pd.DataFrame(errores)


# Cargar dataset y label encoder
from src.fase2.script_1_transformer_preprocessing_optimizado_2 import LightCurveDataset
#torch.serialization.add_safe_globals([LightCurveDataset])

val_dataset = torch.load("../data/train/val_dataset.pt", weights_only=False)

# Verificacion rapida
# Cargar un sample cualquiera
sample = val_dataset[0]
# Ver cuántos elementos contiene
print("Nº de elementos devueltos por __getitem__:", len(sample))
# Inspeccionar los elementos
for i, item in enumerate(sample):
    print(f"Elemento {i}: {type(item)}, shape o valor: {getattr(item, 'shape', item)}")

with open("../data/train/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)
num_classes = len(label_encoder)
class_names = list(label_encoder.keys())

# Dataloader con batch pequeño
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=6, pin_memory=True, persistent_workers=True)

# Crear el modelo con la arquitectura esperada
args = argparse.Namespace(
    input_dim=1,
    in_channels=1,
    encoder_dim=256,
    hidden_dim=384,
    output_dim=num_classes,
    num_heads=8,
    num_layers=8,
    dropout=0.4, dropout_p=0.4,
    stride=32,
    kernel_size=3,
    norm="postnorm",
    encoder=["mhsa_pro", "conv", "conv"],
    timeshift=False,
    device=device
)
model = AstroConformerClassifier(args, num_classes=len(label_encoder), feature_dim=7)

# Cargar los pesos entrenados
state_dict = torch.load("../outputs/mejor_modelo_finetuned_optimizado2_features_segunda_vuelta.pt", map_location="cpu")
# Elimina el prefijo "_orig_mod." de las claves
new_state_dict = {k.replace("_orig_mod.", ""): v for k, v in state_dict.items()}
model.load_state_dict(new_state_dict)

# Pasar a GPU si está disponible
model = model.to(device)
model.eval()  # Muy importante: modo evaluación

criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Evaluar con IDs
val_loss, preds, true, ids = evaluate(model, val_loader, criterion, device)

# Detectar YSO mal clasificadas
df_yso_mal = detectar_yso_confundidas(preds, true, ids, label_encoder)
df_yso_mal.to_csv("../outputs/yso_clase_predicha_error.csv", index=False)

print(f"YSOs mal clasificadas detectadas: {len(df_yso_mal)}")


Nº de elementos devueltos por __getitem__: 5
Elemento 0: <class 'torch.Tensor'>, shape o valor: torch.Size([25000])
Elemento 1: <class 'torch.Tensor'>, shape o valor: torch.Size([])
Elemento 2: <class 'torch.Tensor'>, shape o valor: torch.Size([25000])
Elemento 3: <class 'torch.Tensor'>, shape o valor: torch.Size([7])
Elemento 4: <class 'str'>, shape o valor: AP28318078
YSOs mal clasificadas detectadas: 868


In [ ]:
import pandas as pd

# Cargar CSV de todas las curvas (con clase real)
df_all = pd.read_csv("../data/train/debug_clases_codificadas.csv")

# Cargar errores donde se predijo YSO y no lo era (CASO A)
df_yso_fp = pd.read_csv("../outputs/yso_clase_predicha_error.csv")
ids_fp = set(df_yso_fp["id_objeto"])

# Cargar errores generales de clasificacion (CASO B)
df_errores = pd.read_csv("outputs/errores_mal_clasificados_features_tercera_vuelta.csv")

# Añadir ID si está disponible (debes tener otro CSV con índices → IDs si no lo incluiste)
# Aquí asumimos que ya tienes columna "id_objeto" cruzada
df_fn_yso = df_errores[
    df_errores["clase_real"] == "Young Stellar Object"
].copy()
ids_fn = set(df_fn_yso["id_objeto"])

# 🔁 CURVAS QUE QUEREMOS DEPURAR = unión de ambas
ids_dudosos = ids_fp.union(ids_fn)

# Filtrar todas las YSO reales del dataset original
df_yso_all = df_all[df_all["clase_variable_normalizada"] == "Young Stellar Object"].copy()

# Marcar cuáles son dudosas
df_yso_all["dudosa"] = df_yso_all["id_objeto"].isin(ids_dudosos)

# Estadísticas
print(f"🔢 Total YSO reales: {len(df_yso_all)}")
print(f"⚠️ YSO dudosas detectadas: {df_yso_all['dudosa'].sum()}")

# Ordenar para dejar las seguras primero
df_yso_ordenadas = df_yso_all.sort_values(by="dudosa", ascending=True)

# Seleccionar 9000 curvas más confiables
df_yso_final = df_yso_ordenadas.head(9000).copy()

# Guardar resultado
df_yso_final.to_csv("outputs/ysos_9000_filtradas_seguras.csv", index=False)
print("✅ Guardado en outputs/ysos_9000_filtradas_seguras.csv")
